In [18]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy.integrate import solve_ivp
import copy
import random
import math
import time

from cma import CMA  # Importando a biblioteca CMA-ES
from deap import base, creator, tools, algorithms

from Modules.Helper import Helper
from Modules.Solvers import Solvers
from Modules.Plotters import Plotters
from Modules.Equation import Equation

In [20]:
# Inicialização dos parâmetros
labels = ['A', 'B', 'C', 'D', 'E']
df, max_data = Helper.load_data(filename='Data/GRN5_DATA.txt', labels=labels)
initial_conditions = np.array([df[label].iloc[0] for label in labels])
t_span = (df['t'].iloc[0], df['t'].iloc[-1])
t_eval = np.array(df['t'])
original = np.array(df[labels]).T

# Definindo os limites para os coeficientes
bounds = {
    'tau': (0.1, 10.0),
    'k': (0.1, 5.0),
    'n': (1, 5),  # n deve ser um inteiro, assim a faixa mínima é 1
}

# Tamanho do indivíduo = número de coeficientes
IND_SIZE = 19

# Definindo o criador DEAP
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))  # Minimizar a função de fitness
creator.create("Individual", list, fitness=creator.FitnessMin)

# Classe dos coeficientes
class Coefficient:
    def __init__(self, bounds):
        self.val = random.uniform(*bounds)
        self.bounds = bounds

    def __repr__(self):
        return f"value={self.val}"

# Função para mapear os coeficientes
def map_parameters(x):
    ind = Individual()
    index = 0
    for key, label in ind.coeffs.items():
        label['tau'] = Coefficient(bounds['tau'])  # Instanciando Coefficient para 'tau'
        index += 1
        for key, coeffs in label.items():
            if key != 'tau':
                coeffs['n'] = Coefficient((bounds['n'][0], bounds['n'][1]))  # Instanciando Coefficient para 'n'
                coeffs['k'] = Coefficient((bounds['k'][0], bounds['k'][1]))  # Instanciando Coefficient para 'k'
                index += 1
    return ind

# Classe do indivíduo/solução candidata
class Individual:
    def __init__(self):
        self.coeffs = {
            'A': {
                'E': {'n': None, 'k': None, '-': True},
                'tau': None
            },
            'B': {
                'A': {'n': None, 'k': None, '-': False},
                'tau': None
            },
            'C': {
                'B': {'n': None, 'k': None, '-': False},
                'tau': None,
            },
            'D': {
                'C': {'n': None, 'k': None, '-': False},
                'tau': None,
            },
            'E': {
                'D': {'n': None, 'k': None, '-': False},
                'B': {'n': None, 'k': None, '-': False},
                'E': {'n': None, 'k': None, '-': False},
                'tau': None,
            }
        }
        self.ind_size = IND_SIZE
        self.fitness = np.inf

    # Equações do Modelo GRN5
    @staticmethod
    def system(t, y, equation):
        vals = [Solvers.norm_hardcoded(val, max_data[label]) for val, label in zip(y, labels)]
        N_A, N_B, N_C, N_D, N_E = vals

        dA = equation.full_eq(vals, 'A', 'E')
        dB = equation.full_eq(vals, 'B', 'A')
        dC = equation.full_eq(vals, 'C', 'B')
        dD = equation.full_eq(vals, 'D', 'C')
        dE = equation.complex_eqs(vals, 'E', [['+B', '+D'], ['+D', '+E']])

        return [dA, dB, dC, dD, dE]

    # Cálculo do Erro Absoluto
    @staticmethod
    def absolute_error(original, predicted):
        return sum(sum(abs(original - predicted)))

    # Cálculo do Erro pela distância entre pontos
    @staticmethod
    def squared_error(original, predicted):
        return sum(sum((original - predicted)**2))**(1/2)

    def calculate_fitness(self, method='RK45'):
        equation = Equation(self.numerical_coeffs, labels)
        y = solve_ivp(self.system, t_span, initial_conditions, method=method, t_eval=t_eval, args=(equation, )).y
        self.fitness = self.absolute_error(original, y)

    @property
    def numerical_coeffs(self):
        numerical_coeffs = copy.deepcopy(self.coeffs)
        for key, label in numerical_coeffs.items():
            label['tau'] = label['tau'].val
            for key, coeffs in label.items():
                if key != 'tau':
                    coeffs['n'] = int(coeffs['n'].val)  # Garantindo que n seja um inteiro
                    coeffs['k'] = coeffs['k'].val
        return numerical_coeffs

    def plot(self, method='RK45'):
        methods = [method]
        results = {}
        equation = Equation(self.numerical_coeffs, labels)
        results[method] = solve_ivp(self.system, t_span, initial_conditions, method=method, t_eval=t_eval, args=(equation)).y
        Plotters.plot_methods(results=results, t=t_eval, methods=methods, labels=labels)
        Plotters.plot_comparison(results=results, t=t_eval, df=df, methods=methods, labels=labels)

    def __repr__(self):
        coeffs_repr = {k: v for k, v in self.coeffs.items()}
        return f"Individual(fitness={self.fitness}, coeffs={coeffs_repr}, ind_size={self.ind_size})"

# Função de avaliação
def evaluate(individual):
    individual = map_parameters(individual)  # Mapeia os parâmetros para um objeto Individual
    individual.calculate_fitness()
    return individual.fitness,

# Inicializando o algoritmo evolutivo
population = tools.initRepeat(list, creator.Individual, n=20)  # Inicializa a população com 20 indivíduos

# Função de avaliação para o CMA-ES
def evaluate_cma(x):
    ind = map_parameters(x)  # Mapeia os parâmetros para um objeto Individual
    ind.calculate_fitness()
    return ind.fitness,

# Executando o CMA-ES
x0 = np.random.rand(IND_SIZE)  # Ponto inicial
sigma = 0.5  # Desvio padrão
es = cma.CMAEvolutionStrategy(x0, sigma)

# Loop do CMA-ES
gen_count = 0  # Contador de gerações
while not es.stop():
    solutions = es.ask()
    fitnesses = list(map(evaluate_cma, solutions))
    es.tell(solutions, fitnesses)
    
    # Imprimir a fitness a cada 10 gerações
    gen_count += 1
    if gen_count % 10 == 0:
        print(f"Geração {gen_count}: Melhor fitness = {min(fitnesses)}")

    es.disp()

# Melhor solução encontrada
best_solution = es.result.xbest
print("Melhor solução encontrada:", best_solution)

# Gerar e imprimir o gráfico com os melhores coeficientes
best_individual = map_parameters(best_solution)
best_individual.plot()

(6_w,12)-aCMA-ES (mu_w=3.7,w_1=40%) in dimension 19 (seed=959484, Tue Nov  5 18:08:02 2024)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     12 1.694237829436617e+02 1.0e+00 4.77e-01  5e-01  5e-01 0:00.1
    2     24 1.507933339717443e+02 1.1e+00 4.55e-01  4e-01  5e-01 0:00.2


C:\Users\anabi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
C:\Users\anabi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
C:\Users\anabi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\cma\evolution_strategy.py:2095: UserWarning: function_values is not a list of s

    3     36 1.627952256930087e+02 1.2e+00 4.49e-01  4e-01  5e-01 0:00.2
Geração 10: Melhor fitness = (166.02157942452666,)
Geração 20: Melhor fitness = (181.13483085890758,)
Geração 30: Melhor fitness = (148.02909760840717,)
   38    456 1.686909681204340e+02 2.2e+00 6.75e-01  6e-01  8e-01 0:03.3
Geração 40: Melhor fitness = (165.72933135792434,)
Geração 50: Melhor fitness = (148.03226540060822,)
Geração 60: Melhor fitness = (174.35667847608477,)
Geração 70: Melhor fitness = (152.70639061745592,)
Geração 80: Melhor fitness = (170.3406663194497,)
   88   1056 1.739996661817318e+02 3.0e+00 7.53e-01  6e-01  1e+00 0:07.3
Geração 90: Melhor fitness = (172.7362948648755,)
Geração 100: Melhor fitness = (168.5471844512268,)
  100   1200 1.685471844512268e+02 3.3e+00 1.07e+00  1e+00  2e+00 0:08.2
Geração 110: Melhor fitness = (160.01238983612987,)
Geração 120: Melhor fitness = (155.24447364977206,)
Geração 130: Melhor fitness = (147.85635808772133,)
Geração 140: Melhor fitness = (174.001991266

TypeError: Supplied 'args' cannot be unpacked. Please supply `args` as a tuple (e.g. `args=(<Modules.Equation.Equation object at 0x000001B3ADDE6380>,)`)